# Diabetes Classification using Feedforward Neural Networks
# تصنيف مرض السكري باستخدام الشبكات العصبية التغذوية الأمامية

**Unit:** Unit 5: Introduction to Generative AI and Course Summary  
**Official Structure:** See `../../../DETAILED_UNIT_DESCRIPTIONS.md` for complete requirements

## 📚 Learning Objectives | أهداف التعلم

By completing this notebook, you will:
- Understand binary classification problems
- Learn metrics: Accuracy, Precision, Recall, F1-score
- Perform EDA and data preprocessing for medical datasets
- Build and train FFNN for diabetes classification
- Handle missing data and imbalanced classes

---


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

print("=== Binary Classification Metrics ===")
print("\nMetrics for evaluating binary classification:")
print("  - Accuracy: (TP + TN) / (TP + TN + FP + FN)")
print("  - Precision: TP / (TP + FP) - How many predicted positives are actually positive?")
print("  - Recall: TP / (TP + FN) - How many actual positives did we catch?")
print("  - F1-score: 2 * (Precision * Recall) / (Precision + Recall) - Harmonic mean")


## Example: Diabetes Classification with FFNN


In [ ]:
# Create sample medical dataset (simplified)
np.random.seed(42)
n_samples = 1000

# Features: age, BMI, glucose, blood pressure, etc.
data = {
    'age': np.random.randint(20, 80, n_samples),
    'bmi': np.random.normal(25, 5, n_samples),
    'glucose': np.random.normal(120, 30, n_samples),
    'blood_pressure': np.random.normal(80, 10, n_samples),
    'insulin': np.random.normal(100, 40, n_samples)
}

df = pd.DataFrame(data)

# Create target: diabetes (1) or no diabetes (0)
# Higher glucose and BMI increase diabetes risk
diabetes_prob = 1 / (1 + np.exp(-(df['glucose'] - 120) / 30 - (df['bmi'] - 25) / 5))
df['diabetes'] = (np.random.random(n_samples) < diabetes_prob).astype(int)

print("=== Dataset Overview ===")
print(f"Total samples: {len(df)}")
print(f"Diabetes cases: {df['diabetes'].sum()} ({df['diabetes'].mean():.1%})")
print(f"No diabetes: {(1-df['diabetes']).sum()} ({(1-df['diabetes']).mean():.1%})")
print("\nFirst few rows:")
print(df.head())

# EDA: Check for missing values
print("\n=== EDA: Missing Values ===")
print(df.isnull().sum())

# Prepare data
X = df[['age', 'bmi', 'glucose', 'blood_pressure', 'insulin']].values
y = df['diabetes'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nTraining set: {X_train_scaled.shape[0]} samples")
print(f"Test set: {X_test_scaled.shape[0]} samples")


In [ ]:
# Build FFNN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(5,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=0
)

# Evaluate
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("=== Model Performance ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No Diabetes', 'Diabetes']))
